In [1]:
import ffrd_data_utils as fdu
import xarray as xr

In [2]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33735,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.48 GiB
Comm: tcp://127.0.0.1:36867,Total threads: 3
Dashboard: http://127.0.0.1:39697/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:34989,


In [3]:
TERRAIN = "./data/terrain/Terrain_Addison_M03.cog.tif"
WSEL_TIFS = "./data/wsel_grids/*.tif"
# WSEL_TIFS = "./data/wsel_grids/*.cog.tif"
WSEL_ZARR = "./data/wsel.zarr"
WSEL_RECHUNKED_ZARR = "./data/wsel_rechunked.zarr"
DEPTH_ZARR = "./data/depth.zarr"
# SAMPLE = True
SAMPLE = False

In [4]:
# fdu.tifs_to_zarr(WSEL_TIFS, TERRAIN, WSEL_ZARR, sample=SAMPLE)

In [5]:
# wsel_ds = xr.open_zarr(WSEL_ZARR)
# wsel_ds

In [6]:
# 2 mins 50.4s at 128x128 chunk size 16x16
# 4 mins 12.8s for full domain at chunk size 512x512 with new function
# fdu.wsel_tifs_to_zarr2(WSEL_TIFS, WSEL_ZARR, sample=SAMPLE)

In [7]:
# 1 min 57.5s at 96x512x512 chunk size
# fdu.rechunk_wsel_zarr(WSEL_ZARR, WSEL_RECHUNKED_ZARR)

In [8]:
wsel_ds = xr.open_zarr(WSEL_RECHUNKED_ZARR)
wsel_ds

<xarray.Dataset>
Dimensions:      (r: 3, y: 8899, x: 10249)
Coordinates:
  * r            (r) int64 0 1 2
  * x            (x) float64 1.081e+06 1.081e+06 ... 1.132e+06 1.132e+06
  * y            (y) float64 1.929e+06 1.929e+06 ... 1.885e+06 1.885e+06
Data variables:
    spatial_ref  int64 ...
    terrain      (y, x) float32 dask.array<chunksize=(512, 512), meta=np.ndarray>
    wsel         (r, y, x) float32 dask.array<chunksize=(3, 512, 512), meta=np.ndarray>

In [9]:
# wsel_ds = wsel_ds.chunk({"r": len(wsel_ds.r), "x": fdu.CHUNK_WIDTH, "y": fdu.CHUNK_HEIGHT})
# wsel_ds

In [10]:
depth_da = fdu.depth(wsel_ds.wsel, wsel_ds.terrain)
depth_da

<xarray.DataArray (r: 3, y: 8899, x: 10249)>
dask.array<where, shape=(3, 8899, 10249), dtype=float32, chunksize=(3, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * r        (r) int64 0 1 2
  * x        (x) float64 1.081e+06 1.081e+06 1.081e+06 ... 1.132e+06 1.132e+06
  * y        (y) float64 1.929e+06 1.929e+06 1.929e+06 ... 1.885e+06 1.885e+06

In [ ]:
depth_da = depth_da.chunk({"r": len(depth_da.r), "x": fdu.CHUNK_WIDTH, "y": fdu.CHUNK_HEIGHT})
depth_da

In [11]:
fdu.depth_da_to_zarr(depth_da, DEPTH_ZARR, wsel_ds['spatial_ref'])

In [12]:
depth_ds = xr.open_zarr(DEPTH_ZARR)
depth_ds

<xarray.Dataset>
Dimensions:      (r: 3, y: 8899, x: 10249)
Coordinates:
  * r            (r) int64 0 1 2
  * x            (x) float64 1.081e+06 1.081e+06 ... 1.132e+06 1.132e+06
  * y            (y) float64 1.929e+06 1.929e+06 ... 1.885e+06 1.885e+06
Data variables:
    depth        (r, y, x) float32 dask.array<chunksize=(3, 512, 512), meta=np.ndarray>
    spatial_ref  int64 ...

In [13]:
# depth_ds.depth[2].plot()

In [14]:
# depth_ds = depth_ds.chunk({"r": len(depth_ds.r), "x": fdu.CHUNK_WIDTH, "y": fdu.CHUNK_HEIGHT})
# depth_ds

<xarray.Dataset>
Dimensions:      (r: 3, y: 8899, x: 10249)
Coordinates:
  * r            (r) int64 0 1 2
  * x            (x) float64 1.081e+06 1.081e+06 ... 1.132e+06 1.132e+06
  * y            (y) float64 1.929e+06 1.929e+06 ... 1.885e+06 1.885e+06
Data variables:
    depth        (r, y, x) float32 dask.array<chunksize=(3, 16, 16), meta=np.ndarray>
    spatial_ref  int64 ...

In [15]:
d100 = fdu.depth_quantile(depth_ds.depth, 0.01, spatial_ref=depth_ds.spatial_ref)
d100

<xarray.Dataset>
Dimensions:      (y: 8899, x: 10249)
Coordinates:
  * y            (y) float64 1.929e+06 1.929e+06 ... 1.885e+06 1.885e+06
  * x            (x) float64 1.081e+06 1.081e+06 ... 1.132e+06 1.132e+06
Data variables:
    depth        (y, x) float32 dask.array<chunksize=(16, 16), meta=np.ndarray>
    lower_ci     (y, x) float32 dask.array<chunksize=(16, 16), meta=np.ndarray>
    upper_ci     (y, x) float32 dask.array<chunksize=(16, 16), meta=np.ndarray>
    spatial_ref  int64 ...

In [17]:
d100.to_zarr("./data/d100.zarr", mode="w")

2023-05-19 16:55:21,441 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-05-19 16:55:42,791 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2023-05-19 16:56:36,504 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-05-19 16:57:06,574 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-05-19 16:57:23,359 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-05-19 16:57:43,618 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-05-19 16:58:29,729 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-05-19 16:59:41,662 - distributed.utils_perf - WARNING - full garbage collections took

: 

: 

In [ ]:
# 4 mins 27.2 secs @ 128x128, chunk size 16x16
# d100 = d100.compute()

In [ ]:
d100.depth.plot()

In [ ]:
d100.lower_ci.plot()

In [ ]:
d100.upper_ci.plot()

In [ ]:
(d100.upper_ci - d100.lower_ci).plot()

In [ ]:
wsel_tifs = glob("./data/wsel_grids/*.tif")
wsel_tifs.sort()

wsel_layers = []
for tif in wsel_tifs:
    wsel_ds = rioxarray.open_rasterio(tif, chunks=(1, CHUNK_WIDTH, CHUNK_HEIGHT), masked=True).squeeze(drop=True)
    wsel_ds = wsel_ds[X1:X2, Y1:Y2]
    wsel_layers.append(wsel_ds)

In [ ]:
wsel_layers[0]

In [ ]:
wsel_da = xr.concat(wsel_layers, dim="r")
wsel_da

In [ ]:
wsel_da = wsel_da.chunk({"r": len(wsel_layers), "y": CHUNK_HEIGHT, "x": CHUNK_WIDTH})
wsel_da

In [ ]:
# wsel_da.to_zarr("./data/sample-wsel.zarr", mode="w")

In [ ]:
depths = []
for i in range(0, len(wsel_da.r)):
    depths.append(wsel_da[i] - terrain_da)

depths_da = xr.concat(depths, dim="r")
depths_da = depths_da.chunk({"r": len(wsel_da.r), "y": CHUNK_HEIGHT, "x": CHUNK_WIDTH})
depths_da = depths_da.fillna(0.0)
depths_da

In [ ]:
depths_da.to_zarr("./data/sample-depth.zarr", mode="w")
del(depths_da)

In [ ]:
depths_ds = xr.open_zarr("./data/sample-depth.zarr")
depths_ds

In [ ]:
import numpy as np
from scipy.stats import genextreme as gev, norm

def fit_gev(data: np.ndarray):
    """
    Fit a GEV distribution to the data.
    """
    if np.isnan(data).any():
        return [np.nan, np.nan, np.nan]
    shape, loc, scale = gev.fit(data)
    return [shape, loc, scale]


def fit_norm(data: np.ndarray):
    """
    Fit a normal distribution to the data.
    """
    if np.isnan(data).any():
        return [np.nan, np.nan]
    loc, scale = norm.fit(data)
    return [loc, scale]

In [ ]:
import dask.array as da

# norm_fit_da = da.apply_along_axis(fit_norm, axis=0, arr=depths_da)
# norm_fit_da

In [ ]:
def gev_quantiles(params, probs):
    """
    Compute quantiles from GEV parameters.
    """
    print(params)
    shape, loc, scale = params
    if all(params):
        return gev.ppf(probs, shape, loc=loc, scale=scale)
    bad_result = np.empty(probs.shape)
    bad_result[:] = np.nan
    return bad_result


def norm_quantiles(params, probs):
    """
    Compute quantiles from normal parameters.
    """
    loc, scale = params
    if all(params):
        return norm.ppf(probs, loc=loc, scale=scale)
    bad_result = np.empty(probs.shape)
    bad_result[:] = np.nan
    return bad_result

In [ ]:
# return_periods = [2, 5, 10, 25, 50, 100, 200, 500, 1000]
# quantiles = np.array([1 - (1.0/t) for t in return_periods])
# norm_quantiles = da.apply_along_axis(norm_quantiles, 0, norm_fit_da, quantiles, shape=(len(return_periods),), dtype=np.float64)
# norm_quantiles

In [ ]:
# norm_quantiles_da.compute()

In [ ]:
# norm_quantiles_da = xr.DataArray(norm_quantiles_da, dims=["return_period", "y", "x"], coords={"return_period": return_periods, "y": depths_da.y, "x": depths_da.x})
# norm_quantiles_da

In [ ]:
# norm_quantiles_da.sel(return_period=100).plot()

In [ ]:
# d100 = norm_quantiles_da.sel(return_period=100)
# d100

In [ ]:
# terrain_da_clip = terrain_da[X1:X2, Y1:Y2]
# terrain_da_clip = terrain_da
# terrain_da_clip

In [ ]:
# terrain_da_clip.plot()

In [ ]:
# ws100 = terrain_da_clip + d100
# ws100

In [ ]:
# ws100.plot()

In [ ]:
depths_da = depths_ds.to_array()
depths_da

In [ ]:
def norm_percentile_conf_interval(data: np.ndarray, p: float, n_samples: int = 1000, conf_interval = 90):
    print(data)
    mu, sigma = norm.fit(data)
    percentile = norm.ppf(p, loc=mu, scale=sigma)
    bootstrap_percentiles = []
    for _ in range(n_samples):
        bootstrap_sample = np.random.choice(data, size=data.shape, replace=True)

        mu_bootstrap, sigma_bootstrap = norm.fit(bootstrap_sample)

        bootstrap_percentile = norm.ppf(p, loc=mu_bootstrap, scale=sigma_bootstrap)
        bootstrap_percentiles.append(bootstrap_percentile)
    
    bootstrap_percentiles = np.array(bootstrap_percentiles)
    lower = np.percentile(bootstrap_percentiles, (100 - conf_interval) / 2)
    upper = np.percentile(bootstrap_percentiles, 100 - (100 - conf_interval) / 2)
    return [percentile, lower, upper]

In [ ]:
norm_quantiles_da = da.apply_along_axis(norm_percentile_conf_interval, 1, depths_ds.to_array(), 0.99, shape=(3, ), dtype=np.float64)
norm_quantiles_da

In [ ]:
norm_quantiles_da.compute()

In [ ]:
norm_quantiles_da

In [ ]:
norm_quantiles_da[0].plot()